In [1]:
# visualization_world_map_4countries.py
# Updated world map with 4 countries representing different patterns

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# Set high quality export settings
pio.kaleido.scope.default_width = 1200
pio.kaleido.scope.default_height = 800
pio.kaleido.scope.default_scale = 2

def create_4country_comparison_map(df):
    """Create world map highlighting 4 countries representing different patterns"""
    
    # Filter for latest year
    if 'Year' in df.columns:
        latest_year = df['Year'].max()
        map_data = df[df['Year'] == latest_year].copy()
    else:
        map_data = df.copy()
    
    print(f"🌍 Creating world map with {len(map_data)} countries")
    
    # Define thresholds
    hdi_high_threshold = 0.75  # High HDI threshold
    hdi_low_threshold = 0.55   # Low HDI threshold
    suicide_high_threshold = map_data['Suicide_rate'].quantile(0.75)  # High suicide rate (top 25%)
    suicide_low_threshold = map_data['Suicide_rate'].quantile(0.25)   # Low suicide rate (bottom 25%)
    
    print(f"📊 Thresholds - HDI: High>{hdi_high_threshold:.3f}, Low<{hdi_low_threshold:.3f}")
    print(f"📊 Thresholds - Suicide: High>{suicide_high_threshold:.2f}, Low<{suicide_low_threshold:.2f}")
    
    # Categorize countries into 4 groups
    categories = {
        'High HDI, Low Suicide': {
            'condition': (map_data['HDI'] >= hdi_high_threshold) & (map_data['Suicide_rate'] <= suicide_low_threshold),
            'color': 'green',
            'description': 'Successful Development<br>High development with good mental health'
        },
        'High HDI, High Suicide': {
            'condition': (map_data['HDI'] >= hdi_high_threshold) & (map_data['Suicide_rate'] >= suicide_high_threshold),
            'color': 'red',
            'description': 'Development Paradox<br>High development but mental health challenges'
        },
        'Low HDI, High Suicide': {
            'condition': (map_data['HDI'] <= hdi_low_threshold) & (map_data['Suicide_rate'] >= suicide_high_threshold),
            'color': 'orange',
            'description': 'Double Burden<br>Low development with mental health challenges'
        },
        'Low HDI, Low Suicide': {
            'condition': (map_data['HDI'] <= hdi_low_threshold) & (map_data['Suicide_rate'] <= suicide_low_threshold),
            'color': 'blue',
            'description': 'Resilience<br>Low development but good mental health outcomes'
        }
    }
    
    # Select one representative country from each category
    selected_countries = {}
    country_explanations = {}
    
    for category, config in categories.items():
        category_data = map_data[config['condition']]
        
        if len(category_data) > 0:
            # Select the country closest to the median of its category for better representation
            if 'High HDI' in category:
                # For high HDI groups, prioritize countries with more extreme values
                if 'Low Suicide' in category:
                    # For high HDI, low suicide - pick one with highest HDI
                    selected = category_data.nlargest(3, 'HDI').iloc[0]
                else:
                    # For high HDI, high suicide - pick one with highest suicide rate
                    selected = category_data.nlargest(3, 'Suicide_rate').iloc[0]
            else:
                # For low HDI groups
                if 'Low Suicide' in category:
                    # For low HDI, low suicide - pick one with lowest suicide rate
                    selected = category_data.nsmallest(3, 'Suicide_rate').iloc[0]
                else:
                    # For low HDI, high suicide - pick one with highest suicide rate
                    selected = category_data.nlargest(3, 'Suicide_rate').iloc[0]
            
            selected_countries[category] = selected
            country_explanations[selected['Country Name']] = {
                'category': category,
                'color': config['color'],
                'description': config['description'],
                'details': f"HDI: {selected['HDI']:.3f}, Suicide: {selected['Suicide_rate']:.1f}"
            }
    
    print("🎯 Selected countries for comparison:")
    for category, country in selected_countries.items():
        print(f"   - {category}: {country['Country Name']}")
    
    # Create the world map
    fig = px.choropleth(
        map_data,
        locations="ISO3",
        color="Suicide_rate",
        hover_name="Country Name",
        hover_data={
            'HDI': ':.3f',
            'Suicide_rate': ':.1f',
            'GDP_per_capita': ':,.0f',
            'income_group_auto': True,
            'continent': True
        },
        color_continuous_scale="Reds",
        title="Global Suicide Rates: 4-Country Development-Mental Health Comparison",
        projection="natural earth",
        width=1200,
        height=800
    )
    
    # Highlight selected countries with different colors for each category
    for category, country_data in selected_countries.items():
        config = categories[category]
        fig.add_trace(
            go.Choropleth(
                locations=[country_data['ISO3']],
                z=[10],  # Highlight value
                colorscale=[[0, config['color']], [1, config['color']]],
                showscale=False,
                hoverinfo='skip',
                marker_line_color='black',
                marker_line_width=4,
                name=category
            )
        )
    
    # Create detailed annotations for each country
    annotations = []
    
    # Position annotations in four corners
    annotation_positions = [
        (0.02, 0.98, 'top left'),    # Top left
        (0.98, 0.98, 'top right'),   # Top right  
        (0.02, 0.02, 'bottom left'), # Bottom left
        (0.98, 0.02, 'bottom right') # Bottom right
    ]
    
    for i, (category, country_data) in enumerate(selected_countries.items()):
        if i < len(annotation_positions):
            x_pos, y_pos, anchor = annotation_positions[i]
            config = categories[category]
            
            annotation_text = (
                f"<b>{category}</b><br>"
                f"<b>{country_data['Country Name']}</b><br>"
                f"HDI: {country_data['HDI']:.3f}<br>"
                f"Suicide: {country_data['Suicide_rate']:.1f}<br>"
                f"Income: {country_data.get('income_group_auto', 'N/A')}<br>"
                f"Continent: {country_data.get('continent', 'N/A')}<br>"
                f"<i>{config['description']}</i>"
            )
            
            annotations.append(
                dict(
                    x=x_pos,
                    y=y_pos,
                    xref="paper",
                    yref="paper",
                    text=annotation_text,
                    showarrow=False,
                    bgcolor="white",
                    bordercolor=config['color'],
                    borderwidth=2,
                    borderpad=8,
                    font=dict(size=11, color="black"),
                    align="left"
                )
            )
    
    # Add quadrant explanation
    quadrant_text = (
        "<b>Development-Mental Health Matrix:</b><br>"
        "• 🟢 High HDI, Low Suicide: Successful development<br>"
        "• 🔴 High HDI, High Suicide: Development paradox<br>"  
        "• 🟠 Low HDI, High Suicide: Double burden<br>"
        "• 🔵 Low HDI, Low Suicide: Resilience"
    )
    
    annotations.append(
        dict(
            x=0.5,
            y=0.95,
            xref="paper",
            yref="paper",
            text=quadrant_text,
            showarrow=False,
            bgcolor="white",
            bordercolor="black",
            borderwidth=1,
            borderpad=8,
            font=dict(size=12, color="black"),
            align="center"
        )
    )
    
    fig.update_layout(
        annotations=annotations,
        coloraxis_colorbar=dict(
            title="Suicide Rate<br>(per 100,000)",
            thickness=20,
            len=0.75
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    return fig, selected_countries

def create_comparison_scatter_plot(df, selected_countries):
    """Create a scatter plot highlighting the 4 selected countries"""
    
    # Filter for latest year
    if 'Year' in df.columns:
        latest_year = df['Year'].max()
        plot_data = df[df['Year'] == latest_year].copy()
    else:
        plot_data = df.copy()
    
    # Create the scatter plot
    fig = px.scatter(
        plot_data,
        x='HDI',
        y='Suicide_rate',
        color='continent',
        size='GDP_per_capita',
        hover_name='Country Name',
        hover_data={
            'GDP_per_capita': ':,',
            'Suicide_rate': ':.2f',
            'HDI': ':.3f',
            'income_group_auto': True
        },
        title="HDI vs Suicide Rate: 4-Country Development Patterns Comparison",
        labels={
            'HDI': 'Human Development Index (HDI)',
            'Suicide_rate': 'Suicide Rate (per 100,000)',
            'continent': 'Continent'
        },
        size_max=15,
        opacity=0.6
    )
    
    # Add quadrant lines
    hdi_threshold = 0.65  # Adjusted threshold for better visualization
    suicide_median = plot_data['Suicide_rate'].median()
    
    fig.add_hline(y=suicide_median, line_dash="dash", line_color="gray", opacity=0.7)
    fig.add_vline(x=hdi_threshold, line_dash="dash", line_color="gray", opacity=0.7)
    
    # Highlight the 4 selected countries
    colors = {'High HDI, Low Suicide': 'green', 
              'High HDI, High Suicide': 'red',
              'Low HDI, High Suicide': 'orange', 
              'Low HDI, Low Suicide': 'blue'}
    
    for category, country_data in selected_countries.items():
        fig.add_trace(
            go.Scatter(
                x=[country_data['HDI']],
                y=[country_data['Suicide_rate']],
                mode='markers+text',
                marker=dict(
                    size=20,
                    color=colors[category],
                    symbol='star',
                    line=dict(color='black', width=2)
                ),
                text=country_data['Country Name'],
                textposition="top center",
                name=category,
                hovertemplate=(
                    f"<b>{country_data['Country Name']}</b><br>"
                    f"Category: {category}<br>"
                    f"HDI: {country_data['HDI']:.3f}<br>"
                    f"Suicide Rate: {country_data['Suicide_rate']:.1f}<br>"
                    f"GDP per capita: ${country_data['GDP_per_capita']:,.0f}<br>"
                    f"Income Group: {country_data.get('income_group_auto', 'N/A')}"
                )
            )
        )
    
    # Add quadrant annotations
    quadrants = [
        (0.8, 0.9, "High HDI<br>High Suicide", "red"),
        (0.8, 0.1, "High HDI<br>Low Suicide", "green"),
        (0.3, 0.9, "Low HDI<br>High Suicide", "orange"),
        (0.3, 0.1, "Low HDI<br>Low Suicide", "blue")
    ]
    
    for x, y, text, color in quadrants:
        fig.add_annotation(
            x=x, y=y,
            xref="paper", yref="paper",
            text=text,
            showarrow=False,
            bgcolor=color,
            opacity=0.8,
            font=dict(color="white", size=12),
            bordercolor="black",
            borderwidth=1
        )
    
    fig.update_layout(
        width=1200,
        height=700,
        showlegend=True
    )
    
    return fig

def create_country_comparison_table(selected_countries):
    """Create a detailed comparison table for the 4 countries"""
    
    comparison_data = []
    for category, country_data in selected_countries.items():
        comparison_data.append({
            'Category': category,
            'Country': country_data['Country Name'],
            'HDI': country_data['HDI'],
            'Suicide Rate': country_data['Suicide_rate'],
            'GDP per Capita': f"${country_data['GDP_per_capita']:,.0f}",
            'Income Group': country_data.get('income_group_auto', 'N/A'),
            'Continent': country_data.get('continent', 'N/A')
            # 'Data Quality': country_data.get('Low_data_quality_flag', 'N/A')
        })
    
    df_comparison = pd.DataFrame(comparison_data)
    
    # Create table visualization
    fig = go.Figure(data=[go.Table(
        header=dict(
            values=list(df_comparison.columns),
            fill_color='paleturquoise',
            align='left',
            font=dict(size=12, color='black')
        ),
        cells=dict(
            values=[df_comparison[col] for col in df_comparison.columns],
            fill_color='lavender',
            align='left',
            font=dict(size=11)
        )
    )])
    
    fig.update_layout(
        title="Detailed Comparison: 4-Country Development-Mental Health Profiles",
        width=1200,
        height=400
    )
    
    return fig, df_comparison

def main():
    """Main function to generate the 4-country comparison visualizations"""
    
    print("🚀 Loading data and generating 4-country comparison visualizations...")
    
    # Load data
    try:
        df = pd.read_csv("../Final/final_clean_dataset.csv")
        print(f"✅ Data loaded successfully: {len(df)} rows, {df['Country Name'].nunique()} countries")
    except Exception as e:
        print(f"❌ Error loading data: {e}")
        return
    
    # Generate visualizations
    print("🌍 Generating World Map with 4 Countries...")
    fig_map, selected_countries = create_4country_comparison_map(df)
    fig_map.write_image("world_map_4countries.png", scale=2)
    fig_map.write_html("world_map_4countries.html")
    print("✅ World map saved!")
    
    print("📊 Generating Comparison Scatter Plot...")
    fig_scatter = create_comparison_scatter_plot(df, selected_countries)
    fig_scatter.write_image("scatter_4countries.png", scale=2)
    fig_scatter.write_html("scatter_4countries.html")
    print("✅ Scatter plot saved!")
    
    print("📋 Generating Comparison Table...")
    fig_table, df_table = create_country_comparison_table(selected_countries)
    fig_table.write_image("table_4countries.png", scale=2)
    fig_table.write_html("table_4countries.html")
    df_table.to_csv("comparison_table_4countries.csv", index=False)
    print("✅ Comparison table saved!")
    
    # Print summary
    print("\n🎯 Selected Countries Analysis:")
    print("=" * 60)
    for category, country_data in selected_countries.items():
        print(f"\n{category}:")
        print(f"  Country: {country_data['Country Name']}")
        print(f"  HDI: {country_data['HDI']:.3f}")
        print(f"  Suicide Rate: {country_data['Suicide_rate']:.1f}")
        print(f"  GDP per capita: ${country_data['GDP_per_capita']:,.0f}")
        print(f"  Income Group: {country_data.get('income_group_auto', 'N/A')}")
        print(f"  Continent: {country_data.get('continent', 'N/A')}")
    
    print("\n📁 Files created:")
    print("   - world_map_4countries.png/.html")
    print("   - scatter_4countries.png/.html") 
    print("   - table_4countries.png/.html")
    print("   - comparison_table_4countries.csv")
    
    print("\n🎉 4-country comparison visualizations completed!")

if __name__ == "__main__":
    main()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_23812\652498057.py:11: DeprecationWarning:


Use of plotly.io.kaleido.scope.default_width is deprecated and support will be removed after September 2025.
Please use plotly.io.defaults.default_width instead.


C:\Users\ASUS\AppData\Local\Temp\ipykernel_23812\652498057.py:12: DeprecationWarning:


Use of plotly.io.kaleido.scope.default_height is deprecated and support will be removed after September 2025.
Please use plotly.io.defaults.default_height instead.


C:\Users\ASUS\AppData\Local\Temp\ipykernel_23812\652498057.py:13: DeprecationWarning:


Use of plotly.io.kaleido.scope.default_scale is deprecated and support will be removed after September 2025.
Please use plotly.io.defaults.default_scale instead.




🚀 Loading data and generating 4-country comparison visualizations...
✅ Data loaded successfully: 985 rows, 205 countries
🌍 Generating World Map with 4 Countries...
🌍 Creating world map with 192 countries
📊 Thresholds - HDI: High>0.750, Low<0.550
📊 Thresholds - Suicide: High>12.33, Low<4.89
🎯 Selected countries for comparison:
   - High HDI, Low Suicide: Hong Kong
   - High HDI, High Suicide: Guyana
   - Low HDI, High Suicide: Lesotho
   - Low HDI, Low Suicide: Niger
✅ World map saved!
📊 Generating Comparison Scatter Plot...
✅ Scatter plot saved!
📋 Generating Comparison Table...
✅ Comparison table saved!

🎯 Selected Countries Analysis:

High HDI, Low Suicide:
  Country: Hong Kong
  HDI: 0.955
  Suicide Rate: 4.4
  GDP per capita: $43,748
  Income Group: High
  Continent: Asia

High HDI, High Suicide:
  Country: Guyana
  HDI: 0.776
  Suicide Rate: 31.3
  GDP per capita: $9,889
  Income Group: Upper-Middle
  Continent: America

Low HDI, High Suicide:
  Country: Lesotho
  HDI: 0.550
  Suic